In [3]:
import cv2
import numpy as np
import pandas as pd
import os
import pickle
import time
from datetime import datetime
from deepface import DeepFace

import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog

In [ ]:
# Archivos para almacenar los datos
DATABASE_DIR = "db_faces"
LOG_FILE = "registros_acceso.csv"

def inicializar_sistema():
    if not os.path.exists(DATABASE_DIR):
        os.makedirs(DATABASE_DIR)
    if not os.path.exists(LOG_FILE):
        with open(LOG_FILE, 'w') as f:
            f.write("Fecha,ID_Usuario,Nombre,Estado\n")

def registrar_usuario():
    nombre = simpledialog.askstring("Registro", "Ingrese nombre completo:")
    id_usuario = simpledialog.askstring("Registro", "Ingrese ID único:")
    if not nombre or not id_usuario:
        return

    usuario_dir = os.path.join(DATABASE_DIR, id_usuario)
    if os.path.exists(usuario_dir):
        messagebox.showerror("Error", f"El ID {id_usuario} ya está registrado.")
        return

    os.makedirs(usuario_dir)
    info_usuario = {
        'nombre': nombre,
        'fecha_registro': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    with open(os.path.join(usuario_dir, "info.pkl"), 'wb') as f:
        pickle.dump(info_usuario, f)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Error", "No se pudo acceder a la cámara.")
        return

    capturas = 0
    required_captures = 3

    while capturas < required_captures:
        ret, frame = cap.read()
        if not ret:
            continue

        text = f"Capturas: {capturas}/{required_captures}"
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        try:
            faces = DeepFace.extract_faces(frame, detector_backend='opencv')
            for face in faces:
                if 'facial_area' in face:
                    x = face['facial_area']['x']
                    y = face['facial_area']['y']
                    w = face['facial_area']['w']
                    h = face['facial_area']['h']
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        except:
            pass

        cv2.imshow("Registro de Usuario", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            try:
                faces = DeepFace.extract_faces(frame, detector_backend='opencv')
                if len(faces) != 1:
                    continue
                img_filename = os.path.join(usuario_dir, f"captura_{capturas+1}.jpg")
                cv2.imwrite(img_filename, frame)
                capturas += 1
            except:
                pass

    cap.release()
    cv2.destroyAllWindows()

    if capturas == required_captures:
        img_principal = os.path.join(usuario_dir, "captura_1.jpg")
        try:
            embedding = DeepFace.represent(img_path=img_principal, model_name="VGG-Face", detector_backend="opencv")
            with open(os.path.join(usuario_dir, "embedding.pkl"), 'wb') as f:
                pickle.dump(embedding, f)
            messagebox.showinfo("Éxito", f"Usuario {nombre} registrado con ID: {id_usuario}")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo generar la representación facial: {e}")


def verificar_usuario():
    if not os.path.exists(DATABASE_DIR) or len(os.listdir(DATABASE_DIR)) == 0:
        messagebox.showinfo("Info", "No hay usuarios registrados.")
        return

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Error", "No se pudo acceder a la cámara.")
        return

    db_faces = []
    for user_id in os.listdir(DATABASE_DIR):
        user_dir = os.path.join(DATABASE_DIR, user_id)
        img_path = os.path.join(user_dir, "captura_1.jpg")
        if os.path.exists(img_path):
            db_faces.append((user_id, img_path))

    verified = False
    temp_img_path = "temp_verification.jpg"

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        try:
            faces = DeepFace.extract_faces(frame, detector_backend='opencv')
            for face in faces:
                if 'facial_area' in face:
                    x = face['facial_area']['x']
                    y = face['facial_area']['y']
                    w = face['facial_area']['w']
                    h = face['facial_area']['h']
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        except:
            pass

        cv2.imshow("Verificación", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('v'):
            cv2.imwrite(temp_img_path, frame)
            try:
                faces = DeepFace.extract_faces(frame, detector_backend='opencv')
                if len(faces) == 0:
                    continue

                mejor_coincidencia = None
                menor_distancia = float('inf')

                for user_id, img_path in db_faces:
                    result = DeepFace.verify(
                        img1_path=temp_img_path,
                        img2_path=img_path,
                        model_name="VGG-Face",
                        detector_backend="opencv",
                        distance_metric="cosine"
                    )
                    if result['verified'] and result['distance'] < menor_distancia:
                        menor_distancia = result['distance']
                        mejor_coincidencia = user_id

                os.remove(temp_img_path)

                if mejor_coincidencia:
                    info_path = os.path.join(DATABASE_DIR, mejor_coincidencia, "info.pkl")
                    with open(info_path, 'rb') as f:
                        info_usuario = pickle.load(f)
                    registrar_acceso(mejor_coincidencia, info_usuario['nombre'], True)
                    messagebox.showinfo("Verificación exitosa", f"Bienvenido, {info_usuario['nombre']}\nConfianza: {100 - menor_distancia * 100:.2f}%")
                else:
                    registrar_acceso("desconocido", "No identificado", False)
                    messagebox.showwarning("Fallo", "Verificación fallida.")
            except Exception as e:
                messagebox.showerror("Error", f"Error durante verificación: {e}")
            break
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def registrar_acceso(id_usuario, nombre, acceso_exitoso):
    ahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    estado = "Exitoso" if acceso_exitoso else "Fallido"
    with open(LOG_FILE, 'a') as f:
        f.write(f"{ahora},{id_usuario},{nombre},{estado}\n")

def listar_usuarios():
    if not os.path.exists(DATABASE_DIR):
        messagebox.showinfo("Info", "No hay usuarios registrados.")
        return

    usuarios = []
    for user_id in os.listdir(DATABASE_DIR):
        user_dir = os.path.join(DATABASE_DIR, user_id)
        info_path = os.path.join(user_dir, "info.pkl")
        if os.path.exists(info_path):
            with open(info_path, 'rb') as f:
                info_usuario = pickle.load(f)
            usuarios.append(f"ID: {user_id} - Nombre: {info_usuario['nombre']} - Registro: {info_usuario['fecha_registro']}")

    messagebox.showinfo("Usuarios Registrados", "\n".join(usuarios) if usuarios else "No hay usuarios.")

def eliminar_usuario():
    id_usuario = simpledialog.askstring("Eliminar", "Ingrese el ID del usuario a eliminar:")
    user_dir = os.path.join(DATABASE_DIR, id_usuario)
    if not os.path.exists(user_dir):
        messagebox.showerror("Error", f"No se encontró el ID {id_usuario}.")
        return

    confirm = messagebox.askyesno("Confirmar", f"¿Desea eliminar el usuario {id_usuario}?")
    if confirm:
        for file in os.listdir(user_dir):
            os.remove(os.path.join(user_dir, file))
        os.rmdir(user_dir)
        messagebox.showinfo("Eliminado", f"Usuario {id_usuario} eliminado correctamente.")

def main():
    inicializar_sistema()
    root = tk.Tk()
    root.title("Sistema de Reconocimiento Facial")
    root.geometry("500x400")
    root.configure(bg="#2C3E50")  # Fondo azul oscuro elegante
    
    # Título con estilo
    titulo_frame = tk.Frame(root, bg="#2C3E50")
    titulo_frame.pack(pady=20)
    
    tk.Label(
        titulo_frame, 
        text="Reconocimiento Facial", 
        font=("Helvetica", 22, "bold"),
        fg="#ECF0F1",  # Texto blanco
        bg="#2C3E50"
    ).pack()
    
    tk.Label(
        titulo_frame, 
        text="Powered by DeepFace", 
        font=("Helvetica", 12, "italic"),
        fg="#3498DB",  # Azul claro
        bg="#2C3E50"
    ).pack()
    
    # Contenedor para los botones
    botones_frame = tk.Frame(root, bg="#2C3E50", padx=20)
    botones_frame.pack(pady=20)
    
    # Estilo de botones
    button_style = {
        "font": ("Helvetica", 11),
        "width": 20,
        "borderwidth": 1,
        "relief": tk.RAISED,
        "cursor": "hand2"
    }
    
    # Botones con colores diferentes
    tk.Button(
        botones_frame, 
        text="Registrar Usuario", 
        bg="#2980B9", fg="white",
        activebackground="#3498DB",
        **button_style,
        command=registrar_usuario
    ).pack(pady=8)
    
    tk.Button(
        botones_frame, 
        text="Verificar Identidad", 
        bg="#27AE60", fg="white",
        activebackground="#2ECC71",
        **button_style,
        command=verificar_usuario
    ).pack(pady=8)
    
    tk.Button(
        botones_frame, 
        text="Listar Usuarios", 
        bg="#F39C12", fg="white",
        activebackground="#F1C40F",
        **button_style,
        command=listar_usuarios
    ).pack(pady=8)
    
    tk.Button(
        botones_frame, 
        text="Eliminar Usuario", 
        bg="#E74C3C", fg="white",
        activebackground="#F16745",
        **button_style,
        command=eliminar_usuario
    ).pack(pady=8)
    
    # Botón de salida separado en la parte inferior
    footer_frame = tk.Frame(root, bg="#2C3E50")
    footer_frame.pack(side=tk.BOTTOM, pady=15)
    
    tk.Button(
        footer_frame, 
        text="Salir", 
        bg="#95A5A6", fg="white",
        activebackground="#BDC3C7",
        font=("Helvetica", 10),
        width=10,
        command=root.quit
    ).pack()
    
    # Centra la ventana en la pantalla
    root.update_idletasks()
    width = root.winfo_width()
    height = root.winfo_height()
    x = (root.winfo_screenwidth() // 2) - (width // 2)
    y = (root.winfo_screenheight() // 2) - (height // 2)
    root.geometry('{}x{}+{}+{}'.format(width, height, x, y))
    
    root.mainloop()

if __name__ == "__main__":
    main()

: 

In [3]:
registrar_usuario()

Posicione su rostro frente a la cámara.
Presione 'c' para capturar su rostro (se tomarán 3 capturas).
Captura 1/3 completada.
Captura 2/3 completada.
Captura 3/3 completada.
25-03-21 12:28:51 - vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: C:\Users\jmcr3\.deepface\weights\vgg_face_weights.h5
100%|██████████| 580M/580M [04:35<00:00, 2.11MB/s] 


Usuario Johan registrado exitosamente con ID: 123


In [5]:
verificar_usuario()

Posicione su rostro frente a la cámara.
Presione 'v' para verificar su identidad o 'q' para salir.
Analizando rostro...
Verificación fallida. Usuario no reconocido.


In [6]:
listar_usuarios()


--- USUARIOS REGISTRADOS ---
ID         | Nombre                         | Fecha de Registro   
-----------------------------------------------------------------
123        | Johan                          | 2025-03-21 12:28:19 


In [7]:
eliminar_usuario()

OSError: [WinError 145] El directorio no está vacío: 'db_faces\\'